# Analyzing SASUMO outputs

### Import the Required Libraries

In [ ]:
# %%capture
# !pip install SALib
# !pip install pint-pandas
# !pip install pint
# !pip install plotly
# !pip install lxml

In [ ]:
import os
import sys

In [ ]:
def find_root(path):
    if os.path.split(path)[-1] != 'airport-harper-sumo':
        return find_root(os.path.split(path)[0])
    return path

In [ ]:
PATH = !pwd  # a bit of Jupyter Magic to get the working path
PATH = find_root(PATH[0])
if PATH not in sys.path:
    print(f"adding {PATH} to path")
    sys.path.append(PATH)

# Add savvy to the path
sys.path.append(os.path.join(os.getcwd(), 'savvy',))

In [ ]:
import json5 as json
import sys
import pickle
import csv
import copy

import numpy as np
import pandas as pd 
import pint
import pint_pandas
import plotly.graph_objects as go

from SALib.analyze import sobol
from SASUMO.SASUMO.params import Settings4SASUMO

In [ ]:
ureg = pint.UnitRegistry()

## Defining the Analysis Root

In [ ]:
ROOT = r"/media/HDD/max/sasumo-files/630AM_6_Parameter_Narrow_Dist/2021_09_19-05_09_10"
TOP_LEVEL_SETTINGS = os.path.join(ROOT, "settings.yaml")
OUTPUT= os.path.join(ROOT, "output.txt")
PROBLEM = os.path.join(ROOT, "SALib_Problem.json")

In [ ]:
# with open(TOP_LEVEL_SETTINGS, 'rb') as f:
settings = Settings4SASUMO(TOP_LEVEL_SETTINGS)

## Open up the Problem

In [ ]:
with open(PROBLEM, 'r') as f:
    problem = json.load(f)
problem

## Creating a table of the Results and Parameters

In [ ]:
results = []
param_keys = []
for _dir in os.scandir(ROOT):
    if _dir.is_dir():
        # reading the fuel consumption in 
        try:
            with open(os.path.join(_dir.path, 'f_out.txt'), 'r') as f:
                fc = float(f.read())
        except:
            print(f"issue with {_dir.path}")
        with open(os.path.join(_dir.path, 'sa_values.json'), 'r') as f:
            _d = json.loads(f.read())
            if not param_keys:
                param_keys = list(_d.keys())
            params = [_d[_k] for _k in param_keys]
        results.append([int(_dir.name.split("_")[1]), fc, *params])

In [ ]:
results_df = pd.DataFrame.from_records(results, columns=['Sample_Num', 'f_out', *param_keys],)

# results_df.
results_df.set_index('Sample_Num', inplace=True)
results_df.sort_index(inplace=True)
results_np = results_df['f_out'].values
# results_df.head()

In [ ]:
results_df['Fuel [gal]'] = results_df['f_out'] * (1* ureg.milliliter).to('gallon').magnitude
results_df.head()

## Perform the Sobol Analysis

In [ ]:
Si = sobol.analyze(problem, results_np, print_to_console=False, calc_second_order=True)

## Plotting using Savvy 

In [ ]:
import savvy.interactive_plots as ip
import savvy.network_tools as nt
import savvy.data_processing as dp

from bokeh.plotting import show, output_notebook
output_notebook()

In [ ]:
savvy_d = dp.format_salib_output(Si, run_name='630AM_6_Parameter_Narrow_Dist', pretty_names={'FleetComposition_fleet_composition': "Fleet Composition", "CarFollowingParameters_delta" : "Delta", "PersonalCar_tau": "Tau_PC"})

In [ ]:
# df_list = Si.to_df()
# pretty_names={'FleetComposition_fleet_composition': "Fleet Composition", "CarFollowingParameters_delta" : "Delta", "PersonalCar_tau": "Tau_PC"}

# # combine S1 and ST
# df_list[0] = pd.concat((df_list[0], df_list[1]), axis=1)
# df_list.pop(1)
# df_list[0]['Parameter'] = df_list[0].index

# df_list[0]["Parameter"] = df_list[0]['Parameter'].map(pretty_names).fillna(df_list[0]['Parameter'])
# df_list[0]

In [ ]:
# sa_dict = dp.get_sa_data()
ip.interact_with_plot_all_outputs(savvy_d)

In [ ]:
ip.plot_all_second_order(savvy_d, top=6, mirror=True)

In [ ]:
sa_dict_net = copy.deepcopy(savvy_d)
g = nt.build_graph(savvy_d['630AM_6_Parameter_Narrow_Dist'], sens='ST', top=10, min_sens=1e-6, edge_cutoff=-2, edge_width=100,) #log=True)
nt.plot_network_circle(g, inline=True, scale=100, output_size=(800, 800))
# test = nt.plot_network_random(g, scale=200)

## Plotting the Results

### Surface Plot

In [ ]:
from scipy import interpolate

In [ ]:
x_name = "PersonalCar_tau"
y_name = "PersonalCar_accel"

In [ ]:
series = results_df.groupby(['PersonalCar_tau', 'PersonalCar_accel'])["Fuel [gal]"].median()
series = series.reset_index()
# series.sort_values('FleetComposition_fleet_composition', ascending=False).head(30)

In [ ]:
x = series['FleetComposition_fleet_composition'].values
y = series['Class8Truck_accel'].values
z = series['Fuel [gal]'].values
# X, Y = np.meshgrid(x, y)

In [ ]:
f = interpolate.interp2d(x, y, z, kind="cubic")

In [ ]:
xx = np.linspace(min(x), max(x), 100)
yy = np.linspace(min(y), max(y), 100)
znew = f(xx, yy)

In [ ]:
series['Fuel [gal]'].max()

In [ ]:
fig = go.Figure(data=go.Heatmap(
                    x=xx,
                    y=yy,
                    z=znew,
                    zmin=min(z), 
                    zmax=max(z)
                )
               )

fig.update_layout()
fig.show()

In [ ]:
dict(cmin=min(z), cmax=max(z))

### FC vs Sim #

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Fuel [gal]'], mode='lines'))
fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
                  xaxis=dict(title='Sample Number')
                 )
fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['FleetComposition_fleet_composition'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Fleet Composition [% Truck]')
#                  )
# fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['PersonalCar_tau'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Personal Car Tau [s]')
#                  )
# fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['CarFollowingParameters_delta'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Personal Car Delta')
#                  )
# fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['PersonalCar_accel'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Personal Car Accel [m/s^2]')
#                  )
# fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['Class8Truck_tau'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Class8Truck Tau [s]')
#                  )
# fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['Class8Truck_delta'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Class8Truck Delta')
#                  )
# fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=results_df['Class8Truck_accel'], y=results_df['Fuel [gal]'], mode='markers'))
# fig.update_layout(yaxis=dict(title="Cummulative Fuel Consumption [Gallons]"),
#                   xaxis=dict(title='Class8Truck Accel [m/s^2]')
#                  )
# fig.show()